In [1]:
import pandas as pd

In [2]:
with open("../data/traffic_city.csv", 'r') as file:
    lines=file.readlines()

print(f"The number of rows/lines is {len(lines)}")

The number of rows/lines is 923


In [3]:
print(lines[0]) # column names
print(lines[0].strip('\n').strip().strip(';').split(';')) # columns names as a list

track_id; type; traveled_d; avg_speed; lat; lon; speed; lon_acc; lat_acc; time

['track_id', ' type', ' traveled_d', ' avg_speed', ' lat', ' lon', ' speed', ' lon_acc', ' lat_acc', ' time']


In [4]:
lines_as_lists = [line.strip('\n').strip().strip(';').split(';') for line in lines]
len(lines_as_lists)

923

In [5]:
print(f"the number of fields in row 1 is {len(lines_as_lists[1])}, row 2 is {len(lines_as_lists[2])}")

the number of fields in row 1 is 2710, row 2 is 2680


In [6]:
no_field_max = 0

for row in lines_as_lists:
    if len(row) > no_field_max:
        no_field_max = len(row)

print(f"the maximum number of fields is {no_field_max}")
largest_n = int((no_field_max-4)/6)
print(f"the largest n = {largest_n}")

the maximum number of fields is 122830
the largest n = 20471


In [7]:
cols = lines_as_lists.pop(0)

In [8]:
track_cols = cols[:4]
trajectory_cols = ['track_id'] + cols[4:]

print(track_cols)
print(trajectory_cols)

['track_id', ' type', ' traveled_d', ' avg_speed']
['track_id', ' lat', ' lon', ' speed', ' lon_acc', ' lat_acc', ' time']


In [9]:
track_info = []
trajectory_info = []

for row in lines_as_lists:
    track_id = row[0]

    # add the first 4 values to track_info
    track_info.append(row[:4]) 

    remaining_values = row[4:]
    # reshape the list into a matrix and add track_id
    trajectory_matrix = [ [track_id] + remaining_values[i:i+6] for i in range(0,len(remaining_values),6)]
    # add the matrix rows to trajectory_info
    trajectory_info = trajectory_info + trajectory_matrix


In [10]:
df_track = pd.DataFrame(data= track_info,columns=track_cols)

df_track.head()

track_id         type  traveled_d   avg_speed
0        1          Car       48.85    9.770344
1        2   Motorcycle       98.09   19.839417
2        3   Motorcycle       63.80   18.228752
3        4   Motorcycle      145.72   26.229014
4        5   Motorcycle      138.01   24.841425

In [11]:
df_trajectory = pd.DataFrame(data= trajectory_info,columns=trajectory_cols)

df_trajectory.head()

track_id         lat         lon    speed   lon_acc   lat_acc       time
0        1   37.977391   23.737688   4.9178    0.0518   -0.0299   0.000000
1        1   37.977391   23.737688   4.9207   -0.0124   -0.0354   0.040000
2        1   37.977391   23.737688   4.9160   -0.0519   -0.0413   0.080000
3        1   37.977390   23.737688   4.9057   -0.0914   -0.0478   0.120000
4        1   37.977390   23.737689   4.8871   -0.1679   -0.0550   0.160000

In [12]:
import sys
sys.path.append('../')
from src.database_setup import setup_database

connection_params = {
    "host": "localhost",
    "user": "postgres",
    "password": "postgres",
    "port": "5432",
    "database": "traffic_analysis"
}

# Define your database URL
db_url = f"postgresql://{connection_params['user']}:{connection_params['password']}@{connection_params['host']}:{connection_params['port']}/{connection_params['database']}"

# Call the setup function
setup_database(db_url)



In [13]:
import sqlalchemy
#Loading the dataframes into postgres
engine = sqlalchemy.create_engine(db_url)

df_track.to_sql('track_table', engine, if_exists='replace', index=False)
df_trajectory.to_sql('trajectory_table', engine, if_exists='replace', index=False)


887